In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("learn.csv")

In [3]:
del data["Unnamed: 0"]

In [4]:
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

In [5]:
x = data.copy()

In [6]:
c1 = pd.get_dummies(x.income_type)

In [7]:
x = pd.concat([x, c1], axis = 1)

In [8]:
del x['income_type']

In [9]:
x = x[["application_id", "loan_limit", "loan_rate", "credit_score", "company_enter_month","insert_time",
                  "desired_amount", "EARNEDINCOME", "is_applied"]]

In [11]:
test_data=x[x['insert_time']>='2022-05-16']
train_data = x[x['insert_time']<'2022-05-16']

In [12]:
test_data = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_1885/4276231571.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  test_data = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)


In [13]:
train_data = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_1885/1691262433.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  train_data = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)


In [14]:
test_label = test_data["is_applied"]
del test_data["is_applied"]

In [15]:
train_label = train_data["is_applied"]
del train_data["is_applied"]

In [16]:
from catboost import CatBoostClassifier, Pool

In [17]:
xxx = x.groupby("application_id").max()

In [19]:
k = xxx.is_applied.mean()

In [20]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [21]:
train = Pool(
    data = train_data,
    label = train_label,
    weight = train_label * (1-2*k) + k
)

In [22]:
model.fit(train)

0:	learn: 0.6663350	total: 86.5ms	remaining: 86.5ms
1:	learn: 0.6602777	total: 103ms	remaining: 0us


In [23]:
p = model.predict(test_data)

In [35]:
z = test_data.copy()

In [37]:
z.loc[:, "predict"] = p

In [40]:
appyes = z.loc[z.predict == 1, "application_id"]

In [64]:
zz = data.loc[data.application_id.isin(appyes)]

In [65]:
zz = zz[["application_id", "loan_limit", "loan_rate", "is_applied", "desired_amount"]]

In [66]:
zz = zz.sort_values(by = ["application_id", "loan_rate"])

In [55]:
realdata = x.loc[x.insert_time >= "2022-05-16"]

In [57]:
realdata.loc[:, "predict"] = 0

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_1885/3414315939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  realdata.loc[:, "predict"] = 0


In [58]:
realdata

,application_id,loan_limit,loan_rate,credit_score,company_enter_month,insert_time,desired_amount,EARNEDINCOME,is_applied,predict
8395658,2136969,13000000.0,17.5,650.0,2022-06-01,2022-05-16 00:11:50,2000000.0,0,0.0,0
8395659,2136969,5000000.0,14.5,650.0,2022-06-01,2022-05-16 00:11:50,2000000.0,0,0.0,0
8395660,2136969,3000000.0,12.5,650.0,2022-06-01,2022-05-16 00:11:50,2000000.0,0,1.0,0
8395661,824356,10000000.0,17.5,630.0,2020-10-01,2022-05-16 00:11:51,5000000.0,1,1.0,0
8395662,993637,5000000.0,16.3,700.0,2022-06-01,2022-05-16 00:12:14,1000000.0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...
10264381,455157,3000000.0,18.0,610.0,1997-04-01,2022-05-31 23:54:27,50000000.0,1,0.0,0
10264382,455157,3000000.0,17.5,610.0,1997-04-01,2022-05-31 23:54:27,50000000.0,1,0.0,0
10264383,455157,5000000.0,18.4,610.0,1997-04-01,2022-05-31 23:54:27,50000000.0,1,0.0,0
10264384,455157,3000000.0,14.5,610.0,1997-04-01,2022-05-31 23:54:27,50000000.0,1,0.0,0


In [67]:
zz.loc[:, ["sum", "more"]] = 0

In [68]:
zz.loc[:, "predict"] = 0

In [70]:
zz.iloc[0,5] = zz.iloc[0, 1]

In [75]:
for i in range (1, len(zz)):
    zz.iloc[i, 5] = zz.iloc[i, 1]
    if (zz.iloc[i, 0] == zz.iloc[i-1, 0]):
        zz.iloc[i, 5] += zz.iloc[i-1, 5]

In [76]:
zz["more"] = zz["sum"] > zz.desired_amount

In [77]:
zz

,application_id,loan_limit,loan_rate,is_applied,desired_amount,sum,more,predict
10198527,71,13000000.0,16.6,1.0,5000000.0,13000000,True,0
10052977,92,15000000.0,6.5,0.0,8000000.0,15000000,True,0
10053008,92,15000000.0,6.5,0.0,8000000.0,30000000,True,0
10053121,92,15000000.0,6.5,0.0,8000000.0,45000000,True,0
10052976,92,15000000.0,6.7,0.0,8000000.0,60000000,True,0
...,...,...,...,...,...,...,...,...
9886970,2167746,3000000.0,19.9,0.0,5000000.0,214000000,True,0
9886973,2167746,3000000.0,19.9,0.0,5000000.0,217000000,True,0
8720895,2167790,14000000.0,12.8,0.0,10000000.0,14000000,True,0
8720858,2167790,42000000.0,15.5,0.0,10000000.0,56000000,True,0


In [78]:
for i in range(1, len(zz)):
    if (zz.iloc[i, 0] == zz.iloc[i-1, 0]):
        if (not zz.iloc[i, 6]):
            zz.iloc[i, 7] = 1
        elif (not x.iloc[i-1, 6]):
            zz.iloc[i, 7] = 1
    else:
        zz.iloc[i, 7] = 1

In [79]:
zz.iloc[0, 7] = 1

In [83]:
yesindex = zz.loc[zz.predict == 1].index

In [84]:
yesindex

Int64Index([10198527, 10052977,  9339932,  8900846,  9279078, 10190467,
             9415387,  8624637,  9687876,  8759895,
            ...
             9113556, 10163552,  8710830,  8905506,  9336169,  9882566,
             9902270,  9879289,  9886981,  8720895],
           dtype='int64', length=72081)

In [85]:
realdata.loc[yesindex, "predict"] = 1

In [87]:
from sklearn.metrics import f1_score

In [88]:
f1_score(realdata.is_applied, realdata.predict)

0.3613211924046529

In [89]:
from sklearn.metrics import confusion_matrix

In [91]:
confusion_matrix(realdata.is_applied, realdata.predict).ravel()

array([1722149,   39761,   74498,   32320])